In this notebook, we simulate the field created by the TRIF electromagnet and save it as a data set. This data set is going to be used to test our calibration approach.

The electromagnet has three poles, that we model as three magnetic dipoles oriented in the same direction (physically, each pole is a coil wrapped around a ferromagnetic core). The dipoles are located in the same plain at the same distance from the axis of symmetry, and their position vectors are at $120^0$ with respect to each other.

Here we create a set of various dipole magnitudes and calculate the corresponding magnetic field.

In the experiment each dipole magnitude corresponds to a voltage applied to the corresponding coil inside the magnet. In case of linear relationship between the voltage and magnetic field vector, it is enough to collect and combine the field-vs-voltage characteristic from each individual coil. We'll save this data set separately as linear_calibration_set.csv.

In the non-linear case, in order to properly train an artificial neural network, we'll additionally need data with all coils receiving voltage simultaneously.

Since a constant ambient field can be easily dealt with by subtracting its value from the data set, we do not consider it here.

***

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Functions for the field calculations

def mag(x):
    """Calculates magnitude of vector x"""
    return np.sqrt(np.dot(x,x))

def dipole_field(r, r0, m, a):
    """
    Calculates magnetic field of a single dipole
    r - location of the field
    r0 - position of the dipole
    m - magnitude of the dipole. Negative values correspond to the opposite orientation of  the dipole axis
    a - dipole axis"""
    M = m*a   # Magnetic moment vector
    R = r-r0
    if np.dot(R,R) == 0: #This one is to avoid singularities
        return np.array([0, 0, 0])
    return mu0*(3*R*(np.dot(M,R))/(mag(R)**5) - M/(mag(R)**3))

def total_field(r, m_array: np.array, dipole_axis: np.array) -> np.array:
    """Calculates the total field at point r
    To simplify the data set generation later, the magnitudes of the three dipoles are placed in array m.
    B0 is the external field.
    dipole_axis is the dipole orientation (same for all three)"""
    B1 = dipole_field(r, R1, m_array[0], dipole_axis)
    B2 = dipole_field(r, R2, m_array[1], dipole_axis)
    B3 = dipole_field(r, R3, m_array[2], dipole_axis)
    return B1+B2+B3

In [3]:
# Constants and the setup parameters

mu0 = 10**-7# magnetic constant
m = 1e8 # Scaling factor magnetic dipole magnitude

# Magnetic dipole position vectors
R1 = np.array([1, 0, 1])
R2 = np.array([-np.cos(np.pi/3), np.sin(np.pi/3), 1])
R3 = np.array([-np.cos(np.pi/3), -np.sin(np.pi/3), 1])

# Magnetic dipole orientation (same for all three)
A = np.array([0, 0, 1]) # It is a global variable

# The magnetometer location. Magnetic field is calculated here
r0 = np.array([0., 0., 0.])

# Voltage range
v_range = 10

***

# Linear case

In [4]:
# Apply voltage to a single coil in a linear fashion

N1 = 501 # Number of training samples for each dipole

V1 = np.concatenate((np.linspace(-v_range, v_range, N1), np.zeros(N1), np.zeros(N1)))
V2 = np.concatenate((np.zeros(N1), np.linspace(-v_range, v_range, N1), np.zeros(N1)))
V3 = np.concatenate((np.zeros(N1), np.zeros(N1), np.linspace(-v_range, v_range, N1)))

V_ind = np.stack((V1, V2, V3), axis=1) # Stack vertically
print("V_ind \n=", V_ind)

V_ind 
= [[-10.     0.     0.  ]
 [ -9.96   0.     0.  ]
 [ -9.92   0.     0.  ]
 ...
 [  0.     0.     9.92]
 [  0.     0.     9.96]
 [  0.     0.    10.  ]]


In [5]:
# Calculate magnetic field

B_ind = []
for V in V_ind:
    B_ind.append(total_field(r0, V * m, A))

individual_calibration_set = pd.DataFrame(np.concatenate((B_ind, V_ind), axis = 1), columns = ["B_x", "B_y", "B_z", "V_1", "V_2", "V_3"]) # Combine fields and voltages into a single dataset

In [6]:
individual_calibration_set.head()

,B_x,B_y,B_z,V_1,V_2,V_3
0,-53.033009,0.0,-17.677670,-10.00,0.0,0.0
1,-52.820877,0.0,-17.606959,-9.96,0.0,0.0
2,-52.608745,0.0,-17.536248,-9.92,0.0,0.0
3,-52.396612,0.0,-17.465537,-9.88,0.0,0.0
4,-52.184480,0.0,-17.394827,-9.84,0.0,0.0


In [7]:
# Save
individual_calibration_set.to_csv('data/individual_calibration_set.csv')

***

# Non-linear case
As the non-linearity, we apply a hyperbolic tangent to the voltage before calculating the field. Since in this case we'll need to use deep learning, we generate two sets: one for training and one for validation.


## Training set

In [11]:
nf = 15 # Non-linearity factor

For visualization purposes, we create a set with only one magnetic pole activated at a time, by applying the non-linearity to the linear set.

In [12]:
V_nl_ind = v_range*np.tanh(V_ind/nf)
B_nl_ind = []
for V in V_ind:
    B_nl_ind.append(total_field(r0, V * m, A))

nl_individual_calibration_set = pd.DataFrame(np.concatenate((B_nl_ind, V_nl_ind), axis = 1), columns = ["B_x", "B_y", "B_z", "V_1", "V_2", "V_3"]) # Combine fields and voltages into a single dataset

In [13]:
# Save
nl_individual_calibration_set.to_csv('data/nl_individual_calibration_set.csv')

In [14]:

# Apply random voltages to all three coils
N2 = 2000 # Total number of values
V_rand = v_range*np.tanh(np.random.uniform(-v_range, v_range, (N2, 3))) # Random voltages between -10 and 10
V_full = np.concatenate((V_nl_ind, V_rand)) # stack with the individual voltage set
# Calculate magnetic field

B_full = []
for V in V_full:
    B_full.append(total_field(r0, V * m, A))

full_calibration_set = pd.DataFrame(np.concatenate((B_full, V_full), axis = 1), columns = ["B_x", "B_y", "B_z", "V_1", "V_2", "V_3"]) # Combine fields and voltages into a single dataset
print(full_calibration_set.head())
print(full_calibration_set.tail())

         B_x  B_y        B_z       V_1  V_2  V_3
0 -30.906733  0.0 -10.302244 -5.827829  0.0  0.0
1 -30.813198  0.0 -10.271066 -5.810192  0.0  0.0
2 -30.719373  0.0 -10.239791 -5.792501  0.0  0.0
3 -30.625258  0.0 -10.208419 -5.774754  0.0  0.0
4 -30.530852  0.0 -10.176951 -5.756953  0.0  0.0
            B_x        B_y        B_z       V_1       V_2       V_3
3498 -55.354045  -0.000020  34.581643 -0.437662  9.999994  9.999999
3499 -52.638084 -90.768781 -17.605240 -9.936697 -9.892818  9.870488
3500 -56.873362 -85.194456 -15.117316 -9.999978 -8.550631  9.998964
3501  34.296060 -58.534317  -7.805957  2.839379 -9.999957  2.744861
3502   0.001350  -0.002355  53.032093  9.999997  9.999486  9.999999


In [15]:
# Save
full_calibration_set.to_csv('data/full_calibration_set.csv')

### Validation set for deep learning

We'll use this set to monitor the neural network performance during training in the non-linear case.

In [16]:
N_val = 500

In [17]:

V_val = v_range*np.tanh(np.random.uniform(-v_range, v_range, (N_val, 3))) # Random voltages between -10 and 10
B_val = []

for V in V_val:
    B_val.append(total_field(r0, V * m, A))

val_set = pd.DataFrame(np.concatenate((B_val, V_val), axis = 1), columns = ["B_x", "B_y", "B_z", "V_1", "V_2", "V_3"])

In [18]:
val_set.head()

,B_x,B_y,B_z,V_1,V_2,V_3
0,-52.904322,91.855319,-17.634511,-9.975685,9.999990,-9.999891
1,53.033306,91.855285,17.677455,9.999997,9.999878,-9.999996
2,-106.036466,-0.051155,17.657975,-9.999999,9.988860,9.999998
3,-53.098120,-91.742938,-17.634260,-10.000000,-9.975428,9.999984
4,58.257248,-81.736270,14.149779,9.991503,-9.891907,7.904729


In [19]:
# Save
val_set.to_csv('data/validation_set.csv')